# Proceso de Transformacion de columnas 

Desanidar Columna Genres

In [1]:
# Empiezo importando las librerias que usar en el projecto

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json


In [2]:
# creo un DataFrame en pandas(pd) a partir de los datos contenidos en movies_dataset.csv
pd.set_option('display.max_colwidth', None) # quiero ver los valores completos.
df = pd.read_csv("/Users/benjaminzelaya/Desktop/carpeta/Proyecto_ML_Individual/Datasets Sin Modificar/movies_dataset.csv")

# Imprimo los primeros registros del DataFrame
df.head()

/var/folders/9h/wktndydj5rg6pj7yfx6ws1gm0000gn/T/ipykernel_13171/97459036.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/benjaminzelaya/Desktop/carpeta/Proyecto_ML_Individual/Datasets Sin Modificar/movies_dataset.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.",...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173.0


In [3]:
import ast

# los valores faltantes (NaN) vamos a convertir convirtiéndolos en una lista vacía
df['genres'] = df['genres'].fillna('[]')


In [4]:
# Los datos de cadena en listas de diccionarios los vamos a convertir utilizando 'eval()', ya que no puedo desanidar porque estan en string , y no en diccionarios.

df['genres'] = df['genres'].apply(ast.literal_eval)

In [5]:
# Desanidar la columna production_countries y crear nuevas columnas
df_exploded = df.explode('genres')

def extract_genre_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('id', ''), x.get('name', '')])
    else:
        return pd.Series(['', ''])

df_exploded[['id', 'name']] = df_exploded['genres'].apply(extract_genre_info)

In [6]:
df = df.join(df_exploded[['id','name']], lsuffix='_df', rsuffix='id_genres')


In [7]:
print("Columnas en df_exploded:", df.columns)

Columnas en df_exploded: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'id_df', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'idid_genres', 'name'],
      dtype='object')


In [8]:
df = df.rename(columns={'id_df': 'id'})

df = df.rename(columns={'idid_genres': 'id_genres'})

In [9]:
data_genre = df[['id','name', 'id_genres']]
data_genre 

,id,name,id_genres
0,862,Animation,16
0,862,Comedy,35
0,862,Family,10751
1,8844,Adventure,12
1,8844,Fantasy,14
...,...,...,...
45463,67758,Action,28
45463,67758,Drama,18
45463,67758,Thriller,53
45464,227506,,


In [10]:
# Agrupar por id de películas y crear una lista formateada de name_genres y id_genres
grouped_df = data_genre.groupby('id').agg({'name': lambda x: "'" + "' '".join(x) + "'",
                                           'id_genres': lambda x: "'" + ','.join(map(str, x)) + "'"}).reset_index()

grouped_df

,id,name,id_genres
0,100,'Comedy' 'Crime',"'35,80'"
1,10000,'Comedy' 'Drama',"'35,18'"
2,10001,'Comedy' 'Science Fiction',"'35,878'"
3,100010,'Drama' 'War',"'18,10752'"
4,100017,'Drama','18'
...,...,...,...
45431,99946,'Comedy','35'
45432,9995,'Action' 'Crime' 'Drama',"'28,80,18'"
45433,9997,'Fantasy' 'Horror' 'Action' 'Science Fiction',"'14,27,28,878'"
45434,99977,'Comedy','35'


In [11]:
grouped_df.head()

,id,name,id_genres
0,100,'Comedy' 'Crime',"'35,80'"
1,10000,'Comedy' 'Drama',"'35,18'"
2,10001,'Comedy' 'Science Fiction',"'35,878'"
3,100010,'Drama' 'War',"'18,10752'"
4,100017,'Drama','18'


In [12]:

filtered_movies = grouped_df[grouped_df['id'] == '862']
genres_movie_id2 = filtered_movies['name'].values[0] if len(filtered_movies) > 0 else None

genres_movie_id2 



"'Animation' 'Comedy' 'Family'"

In [18]:
data_genre = grouped_df

# Creo el dataframe con las columnas indicadas guardando el mismo dentro la carpeta ETL/EDA del reposotorio
data_genre.to_csv("genres_groupby.csv", index=False)